## Code to pull pixel brightness values along centerlines

In [20]:
import numpy as np
import pandas as pd   
import os
import matplotlib.pyplot as plt
import matplotlib.pylab as pl
import matplotlib.image as mpimg
import cv2
import PIL

csvpaths = '/home/jukes/Documents/Sample_glaciers/'
basepath = '/media/jukes/jukes1/LS8aws/'
massorsize = "mass"

Grab centerline info and BoxIDs

In [21]:
centerline_df = pd.read_csv(csvpaths+'Boxes_coords_pathrows_SE.csv', sep=',', dtype=str, header=0)
centerline_df = centerline_df.set_index('BoxID')
print(centerline_df.shape)

BoxIDs = list(centerline_df.index)
print(BoxIDs)

centerline_df

(14, 12)
['147', '148', '149', '150', '152', '190', '191', '192', '193', '194', '195', '196', '213', '214']


,lmid50_x,lmid50_y,m50,b50,lmid25_x,lmid25_y,m25,b25,lmid75_x,lmid75_y,m75,b75
BoxID,,,,,,,,,,,,
147,63.0,91.5,0.49230769230769234,60.48461538461538,68.0,81.25,0.5038759689922481,47.77307692307692,58.0,101.75,0.48091603053435117,73.19615384615385
148,68.5,94.0,-0.2535211267605634,111.36619718309859,65.25,80.0,-0.2553191489361702,96.54225352112675,71.75,108.0,-0.2517482517482518,126.19014084507043
149,40.5,71.5,-0.17045454545454544,78.4034090909091,39.75,66.25,-0.16477272727272727,73.02556818181819,41.25,76.75,-0.17613636363636365,83.78125
150,40.0,68.5,0.08791208791208792,64.98351648351648,40.5,62.75,0.07650273224043716,59.18956043956044,39.5,74.25,0.09944751381215469,70.77747252747253
152,63.5,87.0,0.3445378151260504,65.1218487394958,67.75,76.0,0.35319148936170214,52.65756302521008,59.25,98.0,0.3360995850622407,77.58613445378151
190,142.5,219.5,-0.18461538461538463,245.80769230769232,137.75,194.75,-0.18461538461538463,220.18076923076924,147.25,244.25,-0.18461538461538463,271.43461538461537
191,116.5,180.5,-0.021929824561403508,183.0548245614035,116.25,159.75,-0.024229074889867842,162.29934210526315,116.75,201.25,-0.019650655021834062,203.81030701754386
192,47.5,58.5,0.5131578947368421,34.125,50.75,52.25,0.5266666666666666,26.20723684210526,44.25,64.75,0.5,42.04276315789474
193,71.0,155.0,-0.2623762376237624,173.62871287128712,70.5,153.0,-0.36528497409326427,171.49752475247524,71.5,157.0,-0.16824644549763032,175.759900990099


In [31]:
#make lists to store image data and grab image files
ratios = []
imagenames = []

for BoxID in BoxIDs:
    if BoxID == '194':
        imagepath = basepath+"Box"+BoxID+"/resized/"
        #image path
        imgfiles = os.listdir(imagepath)

        #grab terminus box raster
#         tbox = mpimg.imread(csvpaths+"Box"+BoxID+"/rotated_images/final/crop_Box"+BoxID+"_raster_cut.png")
        tbox = mpimg.imread(imagepath+"crop_R_Box"+BoxID+"_cut.png")

        #grab centerline points
        xmin = float(centerline_df.loc[BoxID]['lmid50_x'])
        xmax = np.max(np.array(list(set(np.nonzero(tbox)[0]))))
        c_x = np.linspace(xmin, xmax, int(xmax-xmin))
        c_y = float(centerline_df.loc[BoxID]['m50'])*c_x + float(centerline_df.loc[BoxID]['b50'])

        #load in each image and grab the pixel values indexed by the centerline points:
        for img in imgfiles:
            if img.endswith(BoxID+'.pgm'):

                imagename_pgm = img[:-4]+'.pgm'
                #load in the image using cv2
                image = cv2.imread(imagepath+img,-1)
                imageplt = mpimg.imread(imagepath+img)

                pixelvalues = []
                indexes = []

                #loop through all centerline coordinates
                for i in range(0, len(c_y)):
                    x = int(c_x[i])
                    y = int(c_y[i])
                    pixval = '%.3f' % image[y][x]
                    pixelvalues.append(float(pixval))
                    indexes.append(i)#*15.0)
                    
#                     print(x, y, pixval)

#                 #grab the first and last third
                third = int(len(pixelvalues)/3)
                third1 = pixelvalues[:third]
                third3 = pixelvalues[-third:]
#                 print(third1)
#                 print(third3)
                
#                 fig, ax1 = plt.subplots(figsize=(8,4))
#                 plt.plot(seaice_idxs, seaice_pixvals, 'b--', alpha=0.5)
#                 plt.plot(indexes, pixelvalues, 'k-')
# #                 ax1.set_title(img, fontsize=16)
#                 ax1.set_ylabel('Pixel Value', fontsize=12)
#                 ax1.set_xlabel('Distance along centerline (pixels)', fontsize=12)
#                 
#                 plt.legend(['sea ice', 'open water'])
#                 plt.savefig(csvpaths+'Figures/Centerline_values_example_Box001.png', dpi=500)
#                 plt.show()

                #grab the average of the first and average of the last to compare
                avgthird1 = float(np.average(np.array(third1)))
                avgthird3 = float(np.average(np.array(third3)))
                ratio = "%.2f" % (avgthird1/avgthird3)
                
                ratios.append(ratio)
                imagenames.append(imagename_pgm)
#                 print('Ratio 1st/3rd:', ratio)
#                 print(img)

#                 plt.figure(figsize=(20,10))
#                 imgplt_trim = plt.imshow(cv2.cvtColor(imageplt, cv2.COLOR_BGR2RGB))
#                 plt.plot(c_x, c_y, '--w', linewidth=1.5, alpha=0.3)
#                 plt.text(int(0.02*image.shape[0]), int(0.035*image.shape[1]), ratio
#                          , fontsize=16, color='w')
#                 plt.show()
        
pixels_df = pd.DataFrame(list(zip(imagenames, ratios)))
pixels_df

,0,1
0,crop_R_LC82320162015165LGN00_B8_PS_Buffer194.pgm,2.56
1,crop_R_LC82320162017074LGN00_B8_PS_Buffer194.pgm,1.11
2,crop_R_LC80010162014144LGN00_B8_PS_Buffer194.pgm,1.11
3,crop_R_LC82320162015085LGN00_B8_PS_Buffer194.pgm,1.52
4,crop_R_LC82330162015252LGN00_B8_PS_Buffer194.pgm,1.75
5,crop_R_LC82320162014306LGN00_B8_PS_Buffer194.pgm,1.40
6,crop_R_LC82320162016216LGN00_B8_PS_Buffer194.pgm,2.00
7,crop_R_LC82330162013246LGN00_B8_PS_Buffer194.pgm,1.48
8,crop_R_LC80010162015227LGN00_B8_PS_Buffer194.pgm,1.99
9,crop_R_LC82320162015181LGN00_B8_PS_Buffer194.pgm,2.82


In [49]:
# seaice_pixvals = pixelvalues
# seaice_idxs = indexes

In [32]:
#write to txt file
pixels_df.to_csv(path_or_buf=csvpaths+'Pixelratios_SE.txt', sep=' ', index=False, header=False)